In [1]:
print("hello")

hello


In [2]:
%pwd

'd:\\End-to-End-Medical-chatbot-GenAI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'd:\\End-to-End-Medical-chatbot-GenAI'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# Extract data from the pdf file

def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls= PyPDFLoader)
    documents = loader.load()
    
    return documents

In [10]:
extracted_data = load_pdf_file(data = 'Data/')

In [12]:
#extracted_data

In [16]:
# split the data into chunksText Chunks

def text_split (extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks



In [17]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [21]:
#text_chunks

In [22]:
from langchain.embeddings import HuggingFaceEmbeddings


In [23]:
# Download the embeddings from hugging face

def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [25]:
embeddings = download_hugging_face_embeddings()

C:\Users\vasu\AppData\Local\Temp\ipykernel_14324\3595480879.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name= 'sentence-transformers/all-MiniLM-L6-v2')
c:\Users\vasu\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\vasu\anaconda3\envs\medibot\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files 

In [26]:
query_result = embeddings.embed_query("Hey Vasu")
print("Length", len(query_result))


Length 384


In [33]:
from dotenv import load_dotenv
load_dotenv()

True

In [40]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [54]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key="pcsk_6br9Zm_ALYKeTQRbi2mF5Drz9H4QZHFHYSwamJfV2NABqGrqqVzD5iPQepwVDW47vtZFDW")

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'af78aca6e4e024c722cab73377e2061f', 'Date': 'Tue, 07 Jan 2025 20:21:00 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [55]:
import os
os.environ["PINECONE_API_KEY"] = "pcsk_6br9Zm_ALYKeTQRbi2mF5Drz9H4QZHFHYSwamJfV2NABqGrqqVzD5iPQepwVDW47vtZFDW"

In [60]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

ImportError: cannot import name 'PineconeVectorStore' from 'langchain_pinecone' (c:\Users\vasu\anaconda3\envs\medibot\lib\site-packages\langchain_pinecone\__init__.py)